In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.externals import joblib


def get_data_dir():
    return '../data/'


def merged_file_path():
    return get_data_dir()+'Merged_Data_Excel_Test.xlsx'


def train_file_path():
    return get_data_dir()+'train_data.pkl'


def scaler_file_path():
    return get_data_dir()+'scaler.pkl'


def model_file_path():
    return get_data_dir()+'/models/'+'/model-v4-epoch-145-val_loss-0.00.h5'


df = pd.read_excel(merged_file_path(),index_col=0,parse_dates=True )
df.ffill(inplace=True)
print("Rows in data : "+str(len(df)))

scaler = joblib.load(scaler_file_path())
#w_data = df.as_matrix()

scaled = scaler.transform(df)

timesteps=60
rowcount = scaled.shape[0]
close_price_col_idx=0

X_list=[]
Y_list=[]


for i in range(timesteps,rowcount,1):
    target_row=scaled[i]
    y_val = target_row[close_price_col_idx]
    start_row=i-timesteps
    end_row=i

    Y_list.append(y_val)
    x=scaled[start_row:end_row]
    x_first_row=x[0]
    x_last_row = x[timesteps-1]
    X_list.append(x)


    
X=np.asarray(X_list)
Y_hat=np.asarray(Y_list)

num_features=X.shape[2]


print(X.shape)
print(Y_hat.shape)

print("Data prepared")

model = load_model(model_file_path())

num_samples=X.shape[0]


In [ ]:
close_prices=scaled[timesteps:,close_price_col_idx]

In [ ]:
timestamps=df.index.values[timesteps:]

In [ ]:
cp_df=pd.DataFrame(data=close_prices,columns=['close_price'])


In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import cufflinks as cf

init_notebook_mode(connected=True)
cf.go_offline()

In [ ]:
y_all = model.predict(X, batch_size=32)

In [ ]:
cp_df['forecast']=np.asarray(y_all)

In [ ]:
cp_df['date'] = timestamps
cp_df = cp_df.set_index('date')

In [ ]:
temp_arr=np.zeros((num_samples,num_features))

In [ ]:
temp_arr[:,close_price_col_idx]=cp_df['forecast']

In [ ]:
unscaled=scaler.inverse_transform(temp_arr)

In [ ]:
unscaled_y_out=unscaled[:,close_price_col_idx]

In [ ]:
cp_df['forecast']=unscaled_y_out
cp_df['close_price']=df['Close']

In [ ]:
cp_df.iplot()